## ECE495 Setup Lab: Virtual Environment Test

[Code cred](https://www.nengo.ai/nengo/examples/basic/single-neuron.html)


#### Set up

**Ensure you are using your [495 Virtual Environment](https://github.com/kaitlin-fair/495_setup/) before you begin!**  
    
Import Nengo and other supporting libraries into your program to get started:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nengo
from nengo.dists import Uniform

from urllib.request import urlretrieve
import tensorflow as tf
import nengo_dl

### This is code that you will understand in due time, get excited.

This code is testing that we are set up to program Nengo neurons to represent values over time.

In [ ]:
## Create Neuron
model = nengo.Network(label="A Single Neuron")
with model:
    neuron = nengo.Ensemble(
        1,
        dimensions=1,  # Represent a scalar
        # Set intercept to 0.5
        intercepts=Uniform(-0.5, -0.5),
        # Set the maximum firing rate of the neuron to 100hz
        max_rates=Uniform(100, 100),
        # Set the neuron's firing rate to increase for positive input
        encoders=[[1]],
    )

## Create Input
with model:
    cos = nengo.Node(lambda t: np.cos(8 * t))

## Connect Input to Neuron
with model:
    # Connect the input signal to the neuron
    nengo.Connection(cos, neuron)

## Add probes to view neuron response
with model:
    # The original input
    cos_probe = nengo.Probe(cos)
    # The raw spikes from the neuron
    spikes = nengo.Probe(neuron.neurons)
    # Subthreshold soma voltage of the neuron
    voltage = nengo.Probe(neuron.neurons, "voltage")
    # Spikes filtered by a 10ms post-synaptic filter
    filtered = nengo.Probe(neuron, synapse=0.01)

## Run model
with nengo.Simulator(model) as sim:  # Create the simulator
    sim.run(1)  # Run it for 1 second


### This is where you should edit your code and print your results!

In [ ]:

## Plot the results!
# Plot the decoded output of the ensemble
plt.figure()
plt.plot(sim.trange(), sim.data[filtered])
plt.plot(sim.trange(), sim.data[cos_probe])
plt.xlim(0, 1)
plt.title( "Decoded Neuron Output - LastName" )
plt.show()


### More code that you will understand in due time!

This code is testing that we are set up to perform deep learning with Nengo neurons.

In [ ]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

with nengo.Network(seed=0) as net:

    # NengoDL default parameters for the neurons that will make
    # the training progress more smoothly
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([100])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None
    neuron_type = nengo.LIF(amplitude=0.01)
    nengo_dl.configure_settings(stateful=False)

minibatch_size = 256
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size)